In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from collections import defaultdict, Counter
import json
from glob import glob

In [ ]:
!pip install jamo

In [33]:
import jamo

ModuleNotFoundError: No module named 'jamo'

In [2]:
directory = '라벨링데이터'
dir_list = glob(os.path.join('E:','input','ocr_ancient',directory,'*'))

In [3]:
all_json_list = glob(os.path.join('E:','input','ocr_ancient',directory,'*','*'))

## Class Number

In [4]:
total_class = set()
for i in all_json_list:
    with open(i, 'r', encoding='utf-8-sig') as jfile:
        d = json.load(jfile)
        for j in d['Text_Coord']:
            annotate = j['annotate']
            total_class.add(annotate)

KeyboardInterrupt: 

In [80]:
cnt = 0
for i in sorted([i for i in char_no.items() if i[1] != 0 ], key=lambda x:x[0], reverse=True):
    if i[0]>500:
        cnt += i[1]
cnt

2926

## Background Cleaning

In [69]:
# 'Typeface', 'Info_id', 'Category_name', 'Info_name', 'Period', 'Author', 'Difficulty', 'License_name', 'Info_Data_created', 
# 'Image_id', 'Image_filename', 'Image_Data_captured', 'Image_Width', 'Image_Height', 'Image_dpi', 'Image_Char_no', 'Text_Coord'
w_dict = {k:v for k,v in zip(range(500), [0]*500)}
h_dict = {k:v for k,v in zip(range(500), [0]*500)}
char_no = {k:v for k,v in zip(range(2000), [0]*2000)}
ann_set = set()
cnt = 0
for idx, i in tqdm(enumerate(dir_list)):
#     print(idx)
    dir_name = i.split('\\')[-1]
    tmp_list = glob(os.path.join(i, '*')) # json파일
#     print(dir_name)
    for j in tmp_list:
        with open(j, 'r', encoding='utf-8-sig') as jfile:
            d = json.load(jfile)
#             print(d.keys())
            char_no[d['Image_Char_no']] += 1
            image_name = d['Image_filename']
            image_num = image_name[-4:]

#             fig, ax = plt.subplots(figsize=(12,12))
            image = plt.imread(i.replace('라벨링데이터','이미지데이터')+'\\'+dir_name+image_num+'.png') # h, w, c
#             image_arr = np.fromfile(i.replace('라벨링데이터','이미지데이터')+'\\'+dir_name+image_num+'.png', np.uint8)
#             image = cv2.imdecode(image_arr, cv2.IMREAD_COLOR)
#             ax.imshow(image)
            for k in d['Text_Coord']:
                bbox = k['bbox']
#                 if bbox[2]<56 or bbox[3]<56:
#                     cnt += 1
                w_dict[bbox[2]] += 1
                h_dict[bbox[3]] += 1
                annotate = k['annotate']
                ann_set.add(annotate)
                if len(annotate) == 0:
                    length = len(annotate)
                    for m in range(length):
                        print(annotate[m])
#                 pp = image[bbox[1]:bbox[1]+bbox[3],bbox[0]:bbox[0]+bbox[2],:]
#                 rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3], linewidth=3, edgecolor='r', facecolor='none')
#                 ax.add_patch(rect)
            
#             plt.show()
#             break
    

0it [00:00, ?it/s]

In [22]:
sum(w_dict.values())

5005709

In [41]:
len(ann_set)

4660

In [19]:
total = 0
for i in w_dict:
    if i >= 32:
        total += w_dict[i]
    else:
        continue
total

1220277

In [10]:
total

121197

In [18]:
total = 0
for i in h_dict:
    if i >= 56:
        total += h_dict[i]
    else:
        continue
total

812303

### bbox기준으로 자르고 가로로 긴애들 반자르자.

In [66]:
image_idx,annotation_idx # 세로 긴애들만 417, 자른 후 706 -> 전체 자른 후 23367 이미지

(23367, 23367)

In [ ]:
count = 0

image_idx = 0
annotation_idx = 0

images = [] # images.append({'id':image_idx, 'file_name': jf_name+'/'+jf_name+file_name[-4:]+'.png','width':d['Image_Width'], 'height':d['Image_Height']})
annotations = [] # annotations.append({'id':annotations_idx, 'image_id':image_idx, 'category_id':0,\
                 # 'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})

for idx, i in tqdm(enumerate(dir_list)): # directory
    print(idx, i)
    dir_name = i.split('\\')[-1]
    json_list = glob(os.path.join(i, '*')) # json파일
    for j in json_list:
        with open(j, 'r', encoding='utf-8-sig') as jfile:
            d = json.load(jfile)
            if d['Image_Char_no'] > 400:
                continue
            image_name = d['Image_filename']
            image_num = image_name[-4:]
            
#             fig, ax = plt.subplots(figsize=(8,8))
            image = plt.imread(i.replace('라벨링데이터','이미지데이터')+'\\'+dir_name+image_num+'.png') # h, w, c
#             image = np.zeros((int(d['Image_Height']), int(d['Image_Width']), 3), dtype=int)
            h,w,c = image.shape
            
            ### ----------------- Min, Max 계산 ----------------- ###
            min_y, max_y, min_x, max_x = 1e9, 0, 1e9, 0
            for k in d['Text_Coord']:
                bbox = k['bbox'] # x,y,w,h
                min_y, max_y, min_x, max_x = min(min_y, bbox[1]), max(max_y,bbox[1]+bbox[3]),\
                                            min(min_x, bbox[0]), max(max_x,bbox[0]+bbox[2])
            # 자를 지점 결과
            mini_y, maxi_y, mini_x, maxi_x = max(0,min_y-32), min(h,max_y+32), max(0,min_x-32), min(w, max_x+32)
            
            ### ----------------- 글 주변만 자른 이미지 ----------------- ###
            result = image[mini_y:maxi_y, mini_x:maxi_x,:]
            h_, w_, c_ = result.shape
            
            ### ----------------- 가운데를 자르기 전 빈 공간이 있는지 체크 ----------------- ###
            width_check = np.array([0]*w_)
            
#             ax.imshow(result)
            for k in d['Text_Coord']:
                bbox = k['bbox']
                annotate = k['annotate']
                pp = image[bbox[1]-mini_y:bbox[1]-mini_y+bbox[3],bbox[0]-mini_x:bbox[0]-mini_x+bbox[2],:]
                width_check[bbox[0]-mini_x:bbox[0]-mini_x+bbox[2]] += 1
                rect = patches.Rectangle((bbox[0]-mini_x,bbox[1]-mini_y),bbox[2],bbox[3], linewidth=3, edgecolor='r', facecolor='none')
#                 ax.add_patch(rect)
            
            ### ----------------- 빈 공간이 있으면 이미지 분할 ----------------- ###
            cutting = None
            start = None # 빈공간의 처음지점
            end = None # 빈공간의 마지막 지점 --> 치우치지 않게 자르기 위함
            flag = False
            if h_/w_ < 0.9:
                print('가로가 김!', image_idx)
                for n in range(25, -25,-1):
                    if width_check[w_//2 + n] == 0 and flag == False:
                        start = w_//2 + n
                        end = w_//2 + n
                        flag = True
                    elif width_check[w_//2 + n] == 0 and flag == True:
                        end = w_//2 + n
                    elif width_check[w_//2 + n] != 0 and flag == True:
                        end = w_//2 + n
                        break
                    else:
                        continue
                if start is None or end is None: # 가로가 긴데 못자르면 패스
                    continue
                    
                cutting = (start+end)//2
                
                left_coord = []
                right_coord = []
                
                for k in d['Text_Coord']:
                    bbox = k['bbox']
                    bbox_ = [bbox[0]-mini_x,bbox[1]-mini_y,bbox[2],bbox[3]] # 글자주변제거 annotations 적용
                    if bbox_[0] > cutting:
                        bbox_[0] -= cutting
                        right_coord.append(bbox_)
                    else:
                        left_coord.append(bbox_)

                image_left = result[:,:cutting,:]
                image_right = result[:,cutting:,:]
                h_left, w_left, _ = image_left.shape
                h_right, w_right, _ = image_right.shape
                if 2 > h_left / w_left >= 0.9: # 자른 후 세로가 길어야 저장
#                     plt.imsave(f'E:\\input\\ocr_ancient_aug\\image\\{image_idx:05d}.png')
                    print(image_idx,'-------------------')
                    images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                                   'width':w_left, 'height':h_left})
                    for ann in left_coord:
                        annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                 'bbox':(ann[0],ann[1],ann[2],ann[3]), 'area':ann[2]*ann[3], 'ignore':0,'iscrowd':0,'segmentation':[]})
                        annotation_idx += 1
                    image_idx += 1
                if 2 > h_right / w_right >= 0.9:
#                     plt.imsave(f'E:\\input\\ocr_ancient_aug\\image\\{image_idx:05d}.png')
                    images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                                   'width':w_right, 'height':h_right})
                    for ann in right_coord:
                        annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                                 'bbox':(ann[0]-cutting,ann[1],ann[2],ann[3]), 'area':ann[2]*ann[3], 'ignore':0,'iscrowd':0,'segmentation':[]})
                        annotation_idx += 1
                    image_idx += 1
                
            else: # 세로가 길면
                if h_/w_ > 2 or d['Image_Char_no'] > 200:
                    count += 1
                    continue
                print('세로가 김!', image_idx)

#                 plt.imsave(f'E:\\input\\ocr_ancient_aug\\image\\{image_idx:05d}.png')
                images.append({'id':image_idx, 'file_name': f'{image_idx:05d}.png',\
                               'width':w_, 'height':h_})
                for ann in d['Text_Coord']:
                    seg = []
                    x,y,w,h = ann['bbox'][:4]
                    annotations.append({'id':annotation_idx, 'image_id':image_idx, 'category_id':0,\
                             'bbox':(x-mini_x,y-mini_y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
                    annotation_idx += 1
                image_idx += 1

#             plt.show()
#             break
    

0it [00:00, ?it/s]

0 E:input\ocr_ancient\라벨링데이터\목판본_001_문학류_춘향전_01
세로가 김! 0
1 E:input\ocr_ancient\라벨링데이터\목판본_002_문학류_심청전_01
2 E:input\ocr_ancient\라벨링데이터\목판본_003_문학류_심청전_02
3 E:input\ocr_ancient\라벨링데이터\목판본_005_문학류_심청전_04
세로가 김! 1
4 E:input\ocr_ancient\라벨링데이터\목판본_006_종교서류_능엄경언해3_01
세로가 김! 2
세로가 김! 3
세로가 김! 4
세로가 김! 5
세로가 김! 6
세로가 김! 7
세로가 김! 8
세로가 김! 9
세로가 김! 10
세로가 김! 11
세로가 김! 12
세로가 김! 13
세로가 김! 14
세로가 김! 15
세로가 김! 16
세로가 김! 17
세로가 김! 18
세로가 김! 19
세로가 김! 20
세로가 김! 21
세로가 김! 22
세로가 김! 23
세로가 김! 24
세로가 김! 25
세로가 김! 26
세로가 김! 27
세로가 김! 28
세로가 김! 29
세로가 김! 30
세로가 김! 31
세로가 김! 32
세로가 김! 33
세로가 김! 34
세로가 김! 35
세로가 김! 36
세로가 김! 37
세로가 김! 38
세로가 김! 39
세로가 김! 40
세로가 김! 41
세로가 김! 42
세로가 김! 43
세로가 김! 44
세로가 김! 45
세로가 김! 46
세로가 김! 47
세로가 김! 48
세로가 김! 49
세로가 김! 50
세로가 김! 51
세로가 김! 52
세로가 김! 53
세로가 김! 54
세로가 김! 55
세로가 김! 56
세로가 김! 57
세로가 김! 58
세로가 김! 59
세로가 김! 60
세로가 김! 61
세로가 김! 62
세로가 김! 63
세로가 김! 64
세로가 김! 65
세로가 김! 66
세로가 김! 67
세로가 김! 68
세로가 김! 69
세로가 김! 70
세로가 김! 71
세로가 김! 72
세로가 김! 73
세로가 김! 74
세로가 김! 75
세로가 김!

세로가 김! 613
세로가 김! 614
세로가 김! 615
세로가 김! 616
세로가 김! 617
세로가 김! 618
세로가 김! 619
세로가 김! 620
8 E:input\ocr_ancient\라벨링데이터\목판본_010_종교서류_월인석보15_01
세로가 김! 621
세로가 김! 622
세로가 김! 623
세로가 김! 624
세로가 김! 625
세로가 김! 626
세로가 김! 627
세로가 김! 628
세로가 김! 629
세로가 김! 630
세로가 김! 631
세로가 김! 632
세로가 김! 633
세로가 김! 634
세로가 김! 635
세로가 김! 636
세로가 김! 637
세로가 김! 638
세로가 김! 639
세로가 김! 640
세로가 김! 641
세로가 김! 642
세로가 김! 643
세로가 김! 644
세로가 김! 645
세로가 김! 646
세로가 김! 647
세로가 김! 648
세로가 김! 649
세로가 김! 650
세로가 김! 651
세로가 김! 652
세로가 김! 653
세로가 김! 654
세로가 김! 655
세로가 김! 656
세로가 김! 657
세로가 김! 658
세로가 김! 659
세로가 김! 660
세로가 김! 661
세로가 김! 662
세로가 김! 663
세로가 김! 664
세로가 김! 665
세로가 김! 666
세로가 김! 667
세로가 김! 668
세로가 김! 669
세로가 김! 670
세로가 김! 671
세로가 김! 672
세로가 김! 673
세로가 김! 674
세로가 김! 675
세로가 김! 676
세로가 김! 677
세로가 김! 678
세로가 김! 679
세로가 김! 680
세로가 김! 681
세로가 김! 682
세로가 김! 683
세로가 김! 684
세로가 김! 685
세로가 김! 686
세로가 김! 687
세로가 김! 688
세로가 김! 689
세로가 김! 690
세로가 김! 691
세로가 김! 692
세로가 김! 693
세로가 김! 694
세로가 김! 695
세로가 김! 696
세로가 김! 697
세로가 김! 698
세로

세로가 김! 1307
세로가 김! 1308
세로가 김! 1309
세로가 김! 1310
세로가 김! 1311
세로가 김! 1312
세로가 김! 1313
세로가 김! 1314
세로가 김! 1315
세로가 김! 1316
세로가 김! 1317
세로가 김! 1318
세로가 김! 1319
세로가 김! 1320
세로가 김! 1321
세로가 김! 1322
세로가 김! 1323
세로가 김! 1324
세로가 김! 1325
세로가 김! 1326
세로가 김! 1327
세로가 김! 1328
세로가 김! 1329
세로가 김! 1330
세로가 김! 1331
세로가 김! 1332
세로가 김! 1333
세로가 김! 1334
세로가 김! 1335
세로가 김! 1336
세로가 김! 1337
세로가 김! 1338
13 E:input\ocr_ancient\라벨링데이터\목판본_015_종교서류_석보상절13_01
세로가 김! 1339
세로가 김! 1340
세로가 김! 1341
세로가 김! 1342
세로가 김! 1343
세로가 김! 1344
세로가 김! 1345
세로가 김! 1346
세로가 김! 1347
세로가 김! 1348
세로가 김! 1349
세로가 김! 1350
세로가 김! 1351
세로가 김! 1352
세로가 김! 1353
세로가 김! 1354
세로가 김! 1355
세로가 김! 1356
세로가 김! 1357
세로가 김! 1358
세로가 김! 1359
세로가 김! 1360
세로가 김! 1361
세로가 김! 1362
세로가 김! 1363
세로가 김! 1364
세로가 김! 1365
세로가 김! 1366
세로가 김! 1367
세로가 김! 1368
세로가 김! 1369
세로가 김! 1370
세로가 김! 1371
세로가 김! 1372
세로가 김! 1373
세로가 김! 1374
세로가 김! 1375
세로가 김! 1376
세로가 김! 1377
세로가 김! 1378
세로가 김! 1379
세로가 김! 1380
세로가 김! 1381
세로가 김! 1382
세로가 김! 1383
세로가 김! 1384
세로가 김! 1385

세로가 김! 1962
세로가 김! 1963
세로가 김! 1964
세로가 김! 1965
세로가 김! 1966
세로가 김! 1967
세로가 김! 1968
세로가 김! 1969
세로가 김! 1970
세로가 김! 1971
세로가 김! 1972
세로가 김! 1973
세로가 김! 1974
세로가 김! 1975
세로가 김! 1976
세로가 김! 1977
세로가 김! 1978
세로가 김! 1979
세로가 김! 1980
세로가 김! 1981
세로가 김! 1982
세로가 김! 1983
세로가 김! 1984
세로가 김! 1985
세로가 김! 1986
세로가 김! 1987
세로가 김! 1988
세로가 김! 1989
세로가 김! 1990
세로가 김! 1991
세로가 김! 1992
세로가 김! 1993
세로가 김! 1994
세로가 김! 1995
세로가 김! 1996
세로가 김! 1997
세로가 김! 1998
세로가 김! 1999
세로가 김! 2000
세로가 김! 2001
세로가 김! 2002
세로가 김! 2003
세로가 김! 2004
세로가 김! 2005
세로가 김! 2006
세로가 김! 2007
세로가 김! 2008
세로가 김! 2009
세로가 김! 2010
세로가 김! 2011
세로가 김! 2012
세로가 김! 2013
세로가 김! 2014
세로가 김! 2015
세로가 김! 2016
세로가 김! 2017
세로가 김! 2018
세로가 김! 2019
세로가 김! 2020
세로가 김! 2021
세로가 김! 2022
세로가 김! 2023
세로가 김! 2024
세로가 김! 2025
세로가 김! 2026
세로가 김! 2027
세로가 김! 2028
세로가 김! 2029
20 E:input\ocr_ancient\라벨링데이터\목판본_043_언해류_대학언해_01
세로가 김! 2030
세로가 김! 2031
세로가 김! 2032
세로가 김! 2033
세로가 김! 2034
세로가 김! 2035
세로가 김! 2036
세로가 김! 2037
세로가 김! 2038
세로가 김! 2039
세로가 김! 2040
세로

세로가 김! 2607
세로가 김! 2608
세로가 김! 2609
세로가 김! 2610
세로가 김! 2611
세로가 김! 2612
세로가 김! 2613
세로가 김! 2614
세로가 김! 2615
세로가 김! 2616
세로가 김! 2617
세로가 김! 2618
세로가 김! 2619
세로가 김! 2620
세로가 김! 2621
세로가 김! 2622
세로가 김! 2623
세로가 김! 2624
세로가 김! 2625
세로가 김! 2626
세로가 김! 2627
세로가 김! 2628
세로가 김! 2629
세로가 김! 2630
세로가 김! 2631
세로가 김! 2632
세로가 김! 2633
세로가 김! 2634
세로가 김! 2635
세로가 김! 2636
세로가 김! 2637
세로가 김! 2638
세로가 김! 2639
세로가 김! 2640
세로가 김! 2641
세로가 김! 2642
세로가 김! 2643
세로가 김! 2644
세로가 김! 2645
세로가 김! 2646
세로가 김! 2647
세로가 김! 2648
세로가 김! 2649
세로가 김! 2650
세로가 김! 2651
세로가 김! 2652
세로가 김! 2653
세로가 김! 2654
세로가 김! 2655
세로가 김! 2656
세로가 김! 2657
세로가 김! 2658
세로가 김! 2659
세로가 김! 2660
세로가 김! 2661
세로가 김! 2662
세로가 김! 2663
세로가 김! 2664
세로가 김! 2665
세로가 김! 2666
세로가 김! 2667
세로가 김! 2668
세로가 김! 2669
세로가 김! 2670
세로가 김! 2671
세로가 김! 2672
세로가 김! 2673
세로가 김! 2674
세로가 김! 2675
세로가 김! 2676
세로가 김! 2677
29 E:input\ocr_ancient\라벨링데이터\목판본_053_언해류_시경언해6_01
세로가 김! 2678
세로가 김! 2679
세로가 김! 2680
세로가 김! 2681
세로가 김! 2682
세로가 김! 2683
세로가 김! 2684
세로가 김! 2685
세

세로가 김! 3167
세로가 김! 3168
세로가 김! 3169
세로가 김! 3170
세로가 김! 3171
세로가 김! 3172
세로가 김! 3173
세로가 김! 3174
세로가 김! 3175
세로가 김! 3176
세로가 김! 3177
세로가 김! 3178
세로가 김! 3179
세로가 김! 3180
세로가 김! 3181
세로가 김! 3182
세로가 김! 3183
42 E:input\ocr_ancient\라벨링데이터\목판본_095_기술 및 전문서류_신전자초방언해_01
세로가 김! 3184
세로가 김! 3185
세로가 김! 3186
세로가 김! 3187
세로가 김! 3188
세로가 김! 3189
세로가 김! 3190
세로가 김! 3191
세로가 김! 3192
세로가 김! 3193
세로가 김! 3194
세로가 김! 3195
세로가 김! 3196
세로가 김! 3197
세로가 김! 3198
세로가 김! 3199
세로가 김! 3200
세로가 김! 3201
43 E:input\ocr_ancient\라벨링데이터\목판본_096_기술 및 전문서류_연병지남_01
세로가 김! 3202
세로가 김! 3203
세로가 김! 3204
세로가 김! 3205
세로가 김! 3206
세로가 김! 3207
세로가 김! 3208
세로가 김! 3209
세로가 김! 3210
세로가 김! 3211
세로가 김! 3212
세로가 김! 3213
세로가 김! 3214
세로가 김! 3215
세로가 김! 3216
세로가 김! 3217
세로가 김! 3218
세로가 김! 3219
세로가 김! 3220
세로가 김! 3221
세로가 김! 3222
세로가 김! 3223
세로가 김! 3224
세로가 김! 3225
세로가 김! 3226
세로가 김! 3227
세로가 김! 3228
세로가 김! 3229
세로가 김! 3230
세로가 김! 3231
세로가 김! 3232
세로가 김! 3233
세로가 김! 3234
세로가 김! 3235
세로가 김! 3236
44 E:input\ocr_ancient\라벨링데이터\목판본_097_기술 및 전문

In [117]:
images # 전체 캐릭터 수 포함 : 713, 600이상 제거 : 706, +세로3이상긴것 제거 : 702

[{'id': 0, 'file_name': '00000.png', 'width': 667, 'height': 1003},
 {'id': 1, 'file_name': '00001.png', 'width': 783, 'height': 993},
 {'id': 2, 'file_name': '00002.png', 'width': 1053, 'height': 1061},
 {'id': 3, 'file_name': '00003.png', 'width': 799, 'height': 951},
 {'id': 4, 'file_name': '00004.png', 'width': 798, 'height': 972},
 {'id': 5, 'file_name': '00005.png', 'width': 699, 'height': 921},
 {'id': 6, 'file_name': '00006.png', 'width': 675, 'height': 856},
 {'id': 7, 'file_name': '00007.png', 'width': 757, 'height': 873},
 {'id': 8, 'file_name': '00008.png', 'width': 764, 'height': 902},
 {'id': 9, 'file_name': '00009.png', 'width': 753, 'height': 1051},
 {'id': 10, 'file_name': '00010.png', 'width': 1880, 'height': 2694},
 {'id': 11, 'file_name': '00011.png', 'width': 748, 'height': 1038},
 {'id': 12, 'file_name': '00012.png', 'width': 739, 'height': 1227},
 {'id': 13, 'file_name': '00013.png', 'width': 746, 'height': 964},
 {'id': 14, 'file_name': '00014.png', 'width': 800

## coco format 생성

In [ ]:
coco_dict = dict(
    info= dict(
        year=2021, 
        version="1.0", 
        description="OCR", 
        contributor="", 
        url=None, 
        date_created=None
    ), # year, version, description, contributor, url, date_created
    licenses=dict(
        id=0,
        name='CC BY 4.0',
        url=''
    ), # id, name, url
    images=list(), # id, file_name, height, width
    annotations=list(), # id, image_id, category_id, bbox, area, iscrowd
    categories = [
        dict(id=0,
             name='word')
     ] # id, name, supercategory
)

In [ ]:
# Train coco format
image_idx = 0
annotations_idx = 0

images = []
annotations = []
categories = []
partition = len(json_list)//10
for jf in json_list[:partition*9]:
    jf_name = jf.split('\\')[-2]
    with open(jf, 'rt', encoding='utf-8-sig') as jfile:
        d = json.load(jfile)
        file_name = d['Image_filename']
        images.append({'id':image_idx, 'file_name': jf_name+'/'+jf_name+file_name[-4:]+'.png','width':d['Image_Width'], 'height':d['Image_Height']})
        for ann in d['Text_Coord']:
            x,y,w,h = ann['bbox'][:4]
            annotations.append({'id':annotations_idx, 'image_id':image_idx, 'category_id':0,\
                                'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
            annotations_idx += 1
    image_idx += 1
    
categories.append({'id':0, 'name':'word','super_category':None})

coco_dict['images'] = images
coco_dict['annotations'] = annotations
coco_dict['categories'] = categories

with open('train.json', 'wt', encoding='utf-8') as jfile:
    json.dump(coco_dict, jfile)

In [ ]:
# Valid coco format
image_idx = 0
annotations_idx = 0

images = []
annotations = []
categories = []
partition = len(json_list)//10
for jf in json_list[partition*9:]:
    jf_name = jf.split('\\')[-2]
    with open(jf, 'rt', encoding='utf-8-sig') as jfile:
        d = json.load(jfile)
        file_name = d['Image_filename']
        images.append({'id':image_idx, 'file_name': jf_name+'/'+jf_name+file_name[-4:]+'.png','width':d['Image_Width'], 'height':d['Image_Height']})
        for ann in d['Text_Coord']:
            x,y,w,h = ann['bbox'][:4]
            annotations.append({'id':annotations_idx, 'image_id':image_idx, 'category_id':0,\
                                'bbox':(x,y,w,h), 'area':w*h, 'ignore':0,'iscrowd':0,'segmentation':[]})
            annotations_idx += 1
    image_idx += 1
    
categories.append({'id':0, 'name':'word','super_category':None})

coco_dict['images'] = images
coco_dict['annotations'] = annotations
coco_dict['categories'] = categories

with open('valid.json', 'wt', encoding='utf-8') as jfile:
    json.dump(coco_dict, jfile)